# References

https://docs.docker.com/engine/swarm/stack-deploy/

## Redis Tweaks - Compromise

### @sanjibukai The best solution I've found for 1), and 2) 

> is to simply set them in the compose file:

redis:

    sysctls:
    - net.core.somaxconn=511
    - vm.overcommit_memory=1

and it's my understanding that handling 3) in some way, would require running the container as privileged, giving it access to the host, which isn't the best security idea.

Note: I would hesitate to follow net.core.somaxconn=65535 as the value—far above the 511 Redis wants—without understanding the implications (How TCP Backlog Works in Linux)

https://kb.objectrocket.com/redis/how-to-install-redis-on-ubuntu-using-docker-505



In [1]:
#! mkdir -p code

In [2]:
! ls code

Dockerfile  docker-compose.yml	orig-compose.yaml
Makefile    myapp.py		requirements.txt


# code contains my sample app

In [3]:
# ! (cd code; make test)

In [4]:
#! (cd code; docker-compose down)

In [5]:
! (cd code; docker-compose up -d)


Compose does not use swarm mode to deploy services to multiple nodes in a swarm. All containers will be scheduled on the current node.

To deploy your application across the swarm, use `docker stack deploy`.

Creating network "code_default" with the default driver
Pulling tony (tbutzer/bash-tester:latest)...
latest: Pulling from tbutzer/bash-tester

2d6c1f65: Pulling fs layer
58c5966d: Pulling fs layer
10bd7a65: Pulling fs layer
11956c72: Pulling fs layer
d716637e: Pulling fs layer
46fd4481: Pulling fs layer
Digest: sha256:bed447dc24f009f3accf214b32248000adb07ab3871c8472c07e5f02ebdc63ab 200.7MB/200.7MBB
Status: Downloaded newer image for tbutzer/bash-tester:latest
Building web
Step 1/5 : FROM python:3.4-alpine
3.4-alpine: Pulling from library/python

2f1a9c57: Pulling fs layer
ba2397ef: Pulling fs layer
cf9bbbfd: Pulling fs layer
7e266629: Pulling fs layer
Digest: sha256:c210b660e2ea553a7afa23b41a6ed112f85dbce25cbcb567c75dfe05342a4c4b 1.812MB/1.812MBB
Status: Downloaded newer image fo

In [6]:
! (cd code; make test)

make[1]: Entering directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'
for i in 1 2 3 4 5 6\ ; do\
	curl localhost:8000 ; \
done
Hello World! I have been seen 1 times.
Hello World! I have been seen 2 times.
Hello World! I have been seen 3 times.
Hello World! I have been seen 4 times.
Hello World! I have been seen 5 times.
Hello World! I have been seen 6 times.
make[1]: Leaving directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'


In [7]:
%%bash
cd code ; \
for j in {1..20} ; do \
    make test ; \
done

make[1]: Entering directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'
for i in 1 2 3 4 5 6\ ; do\
	curl localhost:8000 ; \
done
Hello World! I have been seen 7 times.
Hello World! I have been seen 8 times.
Hello World! I have been seen 9 times.
Hello World! I have been seen 10 times.
Hello World! I have been seen 11 times.
Hello World! I have been seen 12 times.
make[1]: Leaving directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'
make[1]: Entering directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'
for i in 1 2 3 4 5 6\ ; do\
	curl localhost:8000 ; \
done
Hello World! I have been seen 13 times.
Hello World! I have been seen 14 times.
Hello World! I have been seen 15 times.
Hello World! I have been seen 16 times.
Hello World! I have been seen 17 times.
Hello World! I have been seen 18 times.
make[1]: Leaving directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'
make[1]: Entering directory '/opt/k8s101/20-docker-swarm-distract

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0  13000      0 --:--:-- --:--:-- --:--:-- 13000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0  13000      0 --:--:-- --:--:-- --:--:-- 13000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0  19500      0 --:--:-- --:--:-- --:--:-- 19500
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0  13333      0 --:--:-- --:--:-- --:--:-- 13333
  % Total    % Received % Xferd  Average Speed   Tim

In [8]:
! (cd code; docker-compose down --volumes)

Stopping code_redis_1 ... 
Stopping code_tony_1  ... 
Stopping code_web_1   ... 
Removing code_redis_1 ... mdone
Removing code_tony_1  ... 
Removing code_web_1   ... 
Removing network code_defaultne


In [9]:
!  docker stack deploy --compose-file code/docker-compose.yml stackdemo

Ignoring unsupported options: build

Creating network stackdemo_default
Creating service stackdemo_redis
Creating service stackdemo_tony
Creating service stackdemo_web


In [10]:
! ls code

Dockerfile  docker-compose.yml	orig-compose.yaml
Makefile    myapp.py		requirements.txt


In [11]:
%%bash

sudo chown "$USER":"$USER" /home/"$USER"/.docker -R
sudo chmod g+rwx "/home/$USER/.docker" -R

chown: cannot access '/home/ubuntu/.docker': No such file or directory
chmod: cannot access '/home/ubuntu/.docker': No such file or directory


CalledProcessError: Command 'b'\nsudo chown "$USER":"$USER" /home/"$USER"/.docker -R\nsudo chmod g+rwx "/home/$USER/.docker" -R\n'' returned non-zero exit status 1.

In [12]:
! docker stack services stackdemo

ID                  NAME                MODE                REPLICAS            IMAGE                        PORTS
76a274vu8e7q        stackdemo_web       replicated          1/1                 tbutzer/stackdemo:latest     *:8000->8000/tcp
io522qsgcy4t        stackdemo_redis     replicated          1/1                 redis:alpine                 
sou2f22ym8gp        stackdemo_tony      replicated          1/1                 tbutzer/bash-tester:latest   


In [13]:
! (cd code; make test)

make[1]: Entering directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'
for i in 1 2 3 4 5 6\ ; do\
	curl localhost:8000 ; \
done
Hello World! I have been seen 1 times.
Hello World! I have been seen 2 times.
Hello World! I have been seen 3 times.
Hello World! I have been seen 4 times.
Hello World! I have been seen 5 times.
Hello World! I have been seen 6 times.
make[1]: Leaving directory '/opt/k8s101/20-docker-swarm-distraction/01-day-123/code'


In [16]:
! curl localhost:8000

Hello World! I have been seen 8 times.


In [17]:
! docker stack rm stackdemo

Removing service stackdemo_redis
Removing service stackdemo_tony
Removing service stackdemo_web
Removing network stackdemo_default


In [18]:
! docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE               PORTS


In [32]:
!  docker stack deploy --compose-file code/docker-compose.yml stackdemo

Ignoring unsupported options: build

Creating network stackdemo_default
Creating service stackdemo_redis
Creating service stackdemo_tony
Creating service stackdemo_web


In [33]:
! docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
5970d9f8ae99        redis:alpine        "docker-entrypoint.s…"   5 seconds ago       Up 3 seconds        6379/tcp            stackdemo_redis.1.o2e5kp2px61n2h34bp9txu1fv


In [34]:
! echo docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` bash

docker exec -it bash


In [31]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` curl web:8000

curl: (6) Could not resolve host: web


https://dev.to/realabbas/explain-redis-like-i-m-five-4anm

In [19]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` redis-cli -h redis ping

PONG


In [ ]:
host 